In [ ]:
import os
import numpy as np
import json
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pathlib
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras import regularizers

with open("food-101/food-101/meta/classes.txt", "r") as f:
    CLASS_NAMES = [item.strip() for item in f]
CLASS_NAMES = np.array(CLASS_NAMES)
classes = CLASS_NAMES
print(len(CLASS_NAMES))

data_dir = 'food-101/food-101/images'
data_dir = pathlib.Path(data_dir)#initialising necessary properties
BATCH_SIZE = 32
IMG_HEIGHT = 299
IMG_WIDTH = 299
STEPS_PER_EPOCH = np.ceil(5000/BATCH_SIZE)

#prepare train data generator with necessary augmentations and validation split
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.25)



#generate training data
print('Train Data')
train_data = train_datagen.flow_from_directory(
    str(data_dir),
    batch_size=BATCH_SIZE,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    classes = list(classes),
    subset='training')

#generate validation data
print('\nValidation Data')
valid_data = train_datagen.flow_from_directory(
    str(data_dir),
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    classes = list(classes),

    subset='validation')



mbv2 = Xception(weights='imagenet', include_top=False, input_shape = (299,299,3))
x = mbv2.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(101,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=mbv2.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.1, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, epochs=20, validation_data=valid_data)

# Step 4: Evaluation
test_loss, test_acc = model.evaluate(valid_data, verbose=2)
print("\nValidation accuracy:", test_acc)
print("\nTest accuracy:", test_acc)

# Step 5: Deployment
# Deploy the trained model in an application where users can upload images and receive predictions of food item labels and calorie content.

101
Train Data
Found 7500 images belonging to 10 classes.

Validation Data
Found 2500 images belonging to 10 classes.


Epoch 1/10
235/235 [==============================] - 344s 1s/step - loss: 1.0047 - accuracy: 0.7001 - val_loss: 0.6000 - val_accuracy: 0.8308
Epoch 2/10
235/235 [==============================] - 318s 1s/step - loss: 0.5028 - accuracy: 0.8621 - val_loss: 0.6160 - val_accuracy: 0.8160
Epoch 3/10
235/235 [==============================] - 339s 1s/step - loss: 0.3631 - accuracy: 0.9005 - val_loss: 0.4982 - val_accuracy: 0.8448
Epoch 4/10
235/235 [==============================] - 322s 1s/step - loss: 0.2819 - accuracy: 0.9249 - val_loss: 0.4887 - val_accuracy: 0.8676
Epoch 5/10
235/235 [==============================] - 321s 1s/step - loss: 0.2285 - accuracy: 0.9397 - val_loss: 0.4503 - val_accuracy: 0.8664
Epoch 6/10
235/235 [==============================] - 321s 1s/step - loss: 0.1941 - accuracy: 0.9485 - val_loss: 0.4580 - val_accuracy: 0.8668
Epoch 7/10
235/235 [==============================] - 320s 1s/step - loss: 0.1741 - accuracy: 0.9535 - val_loss: 0.4950 - val_accuracy: 0.8576

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load a sample image for testing
sample_image_path = 'path/to/sample.jpeg'
img = image.load_img(sample_image_path, target_size=(299, 299))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Rescale the pixel values to [0, 1]

# Use the trained model for prediction
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

class_labels = classes
predicted_class_label = class_labels[predicted_class_index]

print(f"Predicted Class: {predicted_class_label}")


1/1 [==============================] - 0s 24ms/step
Predicted Class: beet_salad
